In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# If running in Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/RecSys_KB/Aleksis

/content/drive/MyDrive/RecSys_KB/Aleksis


In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv', keep_default_na=False)
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [ ]:
pip install sentence_transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
movies.head()

,movieId,title,genres,abstracts,abstracts_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0
4,5,Father of the Bride Part II (1995),Comedy,,0


In [ ]:
for i, row in movies.iterrows():
    if len(row['abstracts']) == 0:
        movies.loc[i, 'abstracts'] = row['title'] + f' genres: {row["genres"]}'

In [ ]:
movies.head()

,movieId,title,genres,abstracts,abstracts_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) genres: Adventure|Animation|C...,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995) genres: Comedy|Drama|...,0
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995) genres: Comedy,0


In [ ]:
model.encode("hello", convert_to_tensor=True, show_progress_bar=True).shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([384])

In [ ]:
abstract_embeddings = model.encode(movies['abstracts'].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

In [ ]:
abstract_embeddings.shape

(9742, 384)

In [ ]:
movies["abstract_embeddings"] = list(abstract_embeddings)
movies.head()

,movieId,title,genres,abstracts,abstracts_count,abstract_ebeddings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) genres: Adventure|Animation|C...,0,"[-0.033013444, -0.020109437, 0.07147088, -0.01..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1,"[-0.011830421, 0.06411612, 0.009135419, -0.134..."
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1,"[-0.04352474, -0.029924009, -0.013244372, -0.0..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995) genres: Comedy|Drama|...,0,"[-0.06438251, -0.06763645, -0.0191318, -0.0043..."
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995) genres: Comedy,0,"[-0.045137625, -0.05005042, -0.015638577, -0.0..."


In [ ]:
movies.to_csv('ml-latest-small/movies.csv', encoding='utf-8', index=False)

In [ ]:
movies.to_csv('movies_with_embeddings.csv', encoding='utf-8', index=False)

In [ ]:
movie2abstract_embed = {}
for i, row in movies.iterrows():
    movie2abstract_embed[row['movieId']] = row['abstract_ebeddings']

In [ ]:
movie2abstract_embed[1]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pickle

In [ ]:
with open('movie2abstract_embed.pickle', 'wb') as f:
    pickle.dump(movie2abstract_embed, f)
    movie2abstract_embed = pickle.load(f)